## Kindle Review Sentiment Analysis

#### About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

#### Best Practises

1. preprocess and cleaning the data
2. Train test split
3. BOW, TF-IDF, Word2vec
4. Train ML model
5. Evaluate the Model

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Load the dataset
import pandas as pd
data=pd.read_csv('Kindle Reviews/all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [23]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'asin', 'helpful', 'rating', 'reviewText',
       'reviewTime', 'reviewerID', 'reviewerName', 'summary',
       'unixReviewTime'],
      dtype='object')

In [24]:
df=data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [25]:
df.shape

(12000, 2)

In [26]:
df.isna().sum()

reviewText    0
rating        0
dtype: int64

In [27]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [28]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## Preprocessing and Cleaning

In [29]:
df['rating']=df['rating'].apply(lambda x: 0 if x<3 else 1)

In [30]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [31]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [32]:
df['reviewText']=df['reviewText'].str.lower()

In [33]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [34]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
from bs4 import BeautifulSoup

In [37]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in set(stopwords.words('english'))]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


### Removing special characters

df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))

#### Breakdown:

- df → Pandas DataFrame

- ['reviewText'] → Selecting the column named reviewText

- .apply() → Apply a function to every row in that column

- lambda x: → Anonymous function where x = each review text

- re.sub(pattern, replace, string) → Replace matching pattern in string




- ^ inside [] → NOT

- a-z → lowercase letters

- A-z → uppercase letters

- 0-9 → numbers

- "-" → hyphen

- "+" → one or more occurrences

### Remove the stopswords

df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))

#### Breakdown:

- x.split() → Split sentence into words

- for y in x.split() → Loop through each word

- if y not in stopwords.words('english') → Keep only words NOT in English stopword list

- " ".join() → Join words back into sentence

### Remove url 

df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

Breakdown:

- r'' → raw string (important for regex)

- (http|https|ftp|ssh) → match these protocols

- :// → literal ://

- Rest part → matches full website link

- '' → replace with empty string

- str(x) → convert to string

### Remove html tags

df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

### Breakdown:

- BeautifulSoup(x, 'lxml') → Parse HTML

- 'lxml' → parser engine

- .get_text() → Extract only visible text

### Remove any additional spaces

df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

### Breakdown:

- x.split() → Removes extra spaces

- " ".join() → Join words with single space

In [38]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


## Lemmatization

In [39]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [40]:
def lemmatizer_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [42]:
df['reviewText']=df['reviewText'].apply(lambda X: lemmatizer_words(X))

In [43]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


## Train Test Split

In [44]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],
                                              test_size=0.20)

## Bag Of Words

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
BOW = CountVectorizer()

In [47]:
X_train_bow=BOW.fit_transform(X_train).toarray()
X_test_bow=BOW.transform(X_test).toarray()

In [48]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(9600, 35963))

## TF-IDF 

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [50]:
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [51]:
X_train_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(9600, 35963))

## Model Trainig using Naive Bayes (GNB)

In [60]:
from sklearn.naive_bayes import GaussianNB
model_bow=GaussianNB()
Model_tfidf=GaussianNB()

In [61]:
# trained model on bow transformed data
model_bow.fit(X_train_bow,y_train)

GaussianNB()

In [62]:
# trained model on tfidf transformed data
Model_tfidf.fit(X_train_tfidf,y_train)

GaussianNB()

## Predictions

In [63]:
y_pred_bow=model_bow.predict(X_test_bow)

In [64]:
y_pred_tfidf=Model_tfidf.predict(X_test_tfidf)

## Evaluating  performance of model

In [65]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

Performance of Model_bow

In [67]:
print('Accuracy of BOW :',accuracy_score(y_test,y_pred_bow))

Accuracy of BOW : 0.5654166666666667


In [68]:
confusion_matrix(y_test,y_pred_bow)

array([[504, 279],
       [764, 853]])

In [70]:
print(classification_report(y_test,y_pred_bow))

              precision    recall  f1-score   support

           0       0.40      0.64      0.49       783
           1       0.75      0.53      0.62      1617

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.56      2400
weighted avg       0.64      0.57      0.58      2400



Performance of  Model_tfidf

In [71]:
print('Accuracy of TF-IDF :',accuracy_score(y_test,y_pred_tfidf))

Accuracy of TF-IDF : 0.56625


In [72]:
confusion_matrix(y_test,y_pred_tfidf)

array([[483, 300],
       [741, 876]])

In [73]:
print(classification_report(y_test,y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.39      0.62      0.48       783
           1       0.74      0.54      0.63      1617

    accuracy                           0.57      2400
   macro avg       0.57      0.58      0.55      2400
weighted avg       0.63      0.57      0.58      2400



To improve accuracy word2vec is good choice